In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import sys
import os
from tqdm import tqdm
from functools import reduce
import joblib
import re

sys.path.append("./tools/")
import funclib

sys.path.append("../")
import benchmark_train as btrain
import benchmark_test as btest
import config as cfg
import benchmark_evaluation as eva

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from pandarallel import pandarallel #  import pandaralle
pandarallel.initialize() # init

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [21]:
t3train = pd.read_excel(cfg.TEMPDIR+'task3train.xlsx')
t3test = pd.read_excel(cfg.TEMPDIR+'task3test.xlsx')

In [26]:
def hanldeec(lsdf):
    counter =0
    reslist =[]
    for item in lsdf.iterrows():
        ecarray=item[1].ec_number.split(',')
        for ec in ecarray:
            reslist = reslist +[[ec.strip(), item[1].valuecounts]]
        counter = counter +1
    res=pd.DataFrame(reslist)
    res_group =res.groupby([0])
    reslist = []
    for x in res_group:
        y = x[1].sum().values
        reslist = reslist +[[y[0], y[1]]]
    res = pd.DataFrame(reslist)
    
    
    return res

def find_all(string,substring):
    return [i for i in range(len(string)) if string.startswith(substring, i)]

In [46]:
train = hanldeec(t3train)
test = hanldeec(t3test)

test['ec']=test[0].apply(lambda x: x[(find_all(x, '.')[-3]-1):])
train['ec']=train[0].apply(lambda x: x[(find_all(x, '.')[-3]-1):])

In [50]:
train.to_csv(cfg.TEMPDIR+'task3train.tsv', sep='\t', index=0)
test.to_csv(cfg.TEMPDIR+'task3test.tsv', sep='\t', index=0)

In [59]:
train['isincomplete']=train.ec.apply(lambda x: 1 if ('-' in x) else 0)
test['isincomplete']=test.ec.apply(lambda x: 1 if ('-' in x) else 0)

In [65]:
train['l1']=train.ec.apply(lambda x: x[0:1])
test['l1']=test.ec.apply(lambda x: x[0:1])

In [79]:
ecgroup  = train.groupby(['l1'])

In [80]:
for gup in ecgroup:
    print(len(gup[1]))

1368
1433
1078
548
247
180


In [81]:
train

,0,1,ec,isincomplete,l1
0,1.-.-.-1.-.-.-1.-.-.-1.-.-.-,939,1.-.-.-,1,1
1,1.1.-.-1.1.-.-,134,1.1.-.-,1,1
2,1.1.1.-1.1.1.-1.1.1.-1.1.1.-1.1.1.-,322,1.1.1.-,1,1
3,1.1.1.11.1.1.11.1.1.11.1.1.11.1.1.11.1.1.1,219,1.1.1.1,0,1
4,1.1.1.101.1.1.10,10,1.1.1.10,0,1
...,...,...,...,...,...
4849,6.5.1.5,1,6.5.1.5,0,6
4850,6.5.1.6,3,6.5.1.6,0,6
4851,6.5.1.7,4,6.5.1.7,0,6
4852,6.6.1.1,45,6.6.1.1,0,6


In [83]:
train[train.ec.isin(list(test.ec))]

,0,1,ec,isincomplete,l1
0,1.-.-.-1.-.-.-1.-.-.-1.-.-.-,939,1.-.-.-,1,1
1,1.1.-.-1.1.-.-,134,1.1.-.-,1,1
2,1.1.1.-1.1.1.-1.1.1.-1.1.1.-1.1.1.-,322,1.1.1.-,1,1
3,1.1.1.11.1.1.11.1.1.11.1.1.11.1.1.11.1.1.1,219,1.1.1.1,0,1
5,1.1.1.1001.1.1.1001.1.1.100,55,1.1.1.100,0,1
...,...,...,...,...,...
4809,6.3.4.16,4,6.3.4.16,0,6
4812,6.3.4.2,561,6.3.4.2,0,6
4835,6.4.1.1,19,6.4.1.1,0,6
4837,6.4.1.36.4.1.3,22,6.4.1.3,0,6


In [87]:
937-639-41

257

In [88]:
639+41+257

937

In [90]:
t3train

,ec_number,valuecounts
0,1.-.-.-,930
1,1.1.-.-,133
2,1.1.1.-,311
3,"1.1.1.-,4.2.1.107,4.2.1.119",3
4,"1.1.1.-,4.2.1.119",4
...,...,...
5410,6.5.1.5,1
5411,6.5.1.6,3
5412,6.5.1.7,4
5413,6.6.1.1,45
